# Demo 1 — "Living Scenario Brief"
## Adaptive Situational Awareness Under Uncertainty

> **Responsible-AI Scope Statement:** This demo uses **fully synthetic** scenario data—fictional actors, place-names, and intelligence reports—to illustrate multi-agent orchestration patterns. No output constitutes real intelligence, operational guidance, or doctrinal authority. A human decision-maker retains full authority over any real-world action.

**OODA Phase: Observe / Orient**

**Purpose:** A multi-agent scenario-learning loop that maintains a "living world state" — a structured JSON of events, actors, locations, and uncertainties — and updates it each turn as new simulated intelligence arrives. Each cycle produces a structured SITREP explaining *what* the current situation is and *how and why* it changed.

**Audience:** Warfighters, operational planners, and C2 staff familiar with the intelligence cycle.
**Primary outcome:** The audience sees how multi-agent orchestration automates intelligence fusion, assessment tracking, and briefing generation — the OODA Observe/Orient phases — while keeping the commander in the loop.


## What It Illustrates (Multi-Agent)

| Agent | Role | OODA Phase |
|-------|------|------------|
| **Scenario Orchestrator** | Maintains authoritative world-state JSON; integrates new intel each turn | Foundation |
| **ISR / Intel Fusion** | Fuses multi-INT reports; flags contradictions, corroborations, and gaps | Observe |
| **Assessment Agent** | Updates threat/risk ratings with Bayesian-inspired confidence tracking | Orient |
| **Briefing / Explainer** | Produces commander-ready SITREP in military format with evidence citations | Orient → Brief |
| **Commander (Human-in-the-Loop)** | Asks clarifying questions, challenges assessments, injects CCIRs | Decide |

**Architecture:** AutoGen 0.7 `RoundRobinGroupChat` with four `AssistantAgent`s sharing one `model_client`, streamed via `Console`.

**Success criteria:** After 3 turns of escalating intelligence, the system produces increasingly detailed SITREPs that track confidence changes, cite evidence, identify information gaps, and respond to commander queries — all visible in the agent conversation flow.


## Azure Technologies Used in This Demo

This demo relies on several Azure services working together. If you're new to Azure, here's a quick guide to each technology and how it fits in.

### Azure AI Foundry (formerly Azure AI Studio)

[Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/what-is-ai-foundry) is Microsoft's unified platform for building, evaluating, and deploying generative AI applications. It provides a **project workspace** where you can:

- Access multiple large language models (LLMs) from a single endpoint — including GPT-4o, Mistral, Phi, and more
- Manage prompt engineering, evaluations, and deployments
- Monitor usage, cost, and safety metrics

In this demo, Azure AI Foundry hosts the LLM that powers all four agents. The notebook connects via an **inference endpoint** (a URL) and an **API key** (a credential), both stored securely in Azure Key Vault.

> **Learn more:** [Get started with Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/quickstarts/get-started-playground) | [Azure AI Foundry SDKs](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/sdk-overview)

---

### Azure Key Vault

[Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) is a cloud service for securely storing and managing **secrets** (API keys, passwords, certificates, and cryptographic keys). Instead of pasting credentials directly into code — which risks accidental exposure through version control — Key Vault provides:

- **Centralized secret management** with fine-grained access control (RBAC)
- **Audit logging** of every secret access via Azure Monitor
- **Automatic rotation and expiration** policies
- **FIPS 140-2 validated** hardware-backed storage (HSM tier available)

This project stores the Azure AI Foundry endpoint URL and API key as Key Vault secrets. The code retrieves them at runtime using `DefaultAzureCredential` — so **no credential ever appears in the codebase**.

> **Learn more:** [About Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) | [Quickstart: Set and retrieve a secret](https://learn.microsoft.com/azure/key-vault/secrets/quick-create-portal)

---

### Azure Identity & `DefaultAzureCredential`

The [`azure-identity`](https://learn.microsoft.com/python/api/overview/azure/identity-readme) library provides a unified way to authenticate with Azure services. The star of the library is **`DefaultAzureCredential`**, which tries a chain of authentication methods automatically:

1. **Environment variables** — checks for `AZURE_CLIENT_ID`, `AZURE_TENANT_ID`, `AZURE_CLIENT_SECRET`
2. **Managed Identity** — when running on Azure (VMs, App Service, Azure ML compute), uses the resource's built-in identity with no credentials needed
3. **Azure CLI** — uses your `az login` session during local development
4. **Visual Studio Code** — uses your signed-in Azure account in VS Code

This "just works" pattern means the same code runs locally (using your `az login`) and in Azure (using Managed Identity) without any code changes.

> **Learn more:** [DefaultAzureCredential overview](https://learn.microsoft.com/python/api/overview/azure/identity-readme#defaultazurecredential) | [Azure Identity client library](https://learn.microsoft.com/python/api/overview/azure/identity-readme)

---

### Azure AI Inference SDK

The [`azure-ai-inference`](https://learn.microsoft.com/python/api/overview/azure/ai-inference-readme) package provides a Python client for calling models deployed on Azure AI Foundry. AutoGen 0.7 wraps this via `AzureAIChatCompletionClient` (from `autogen-ext[azure]`), which handles:

- Constructing chat completion requests with system/user messages
- Streaming responses token-by-token
- Retry logic and error handling

The `AzureKeyCredential` class (from `azure.core.credentials`) wraps your API key for authenticated requests.

> **Learn more:** [Azure AI Inference client library](https://learn.microsoft.com/python/api/overview/azure/ai-inference-readme) | [azure.core.credentials](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential)

---

### AutoGen 0.7 (Microsoft Research)

[AutoGen](https://microsoft.github.io/autogen/stable/) is an open-source framework from Microsoft Research for building **multi-agent AI applications**. Key concepts used in this demo:

| Concept | What it does | Docs |
|---------|-------------|------|
| **`AssistantAgent`** | An agent with a system prompt and an LLM `model_client`. It receives messages, reasons via the LLM, and responds. | [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) |
| **`RoundRobinGroupChat`** | A team orchestration pattern where agents take turns in a fixed order (Agent 1 → 2 → 3 → 4). | [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat) |
| **`MaxMessageTermination`** | A stop condition that ends the group chat after a set number of messages. | [Termination Conditions](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html) |
| **`Console`** | A utility that streams agent messages to the notebook output in real time. | [Console](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.ui.html) |
| **`model_client`** | The LLM connection object (here, `AzureAIChatCompletionClient`) shared by all agents. | [Azure model client](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.azure.html) |

AutoGen's value is that you define each agent's role via a system prompt, wire them into a team, and the framework handles message passing, turn-taking, and termination — letting you focus on the problem, not the plumbing.

> **Learn more:** [AutoGen documentation](https://microsoft.github.io/autogen/stable/) | [AutoGen GitHub](https://github.com/microsoft/autogen)


## Demo Script (Presenter Guide)

1. **Intro (1 min):** "Every warfighter knows the SITREP. This demo automates the intelligence fusion behind it using four cooperating AI agents. All scenario content is fully synthetic — no real data or units."
2. **Config (30 sec):** Point out the LLM config, scenario selection, and world-state JSON. "This is the shared data backbone all five demos reuse. Note the synthetic data marker."
3. **Turn 1 (2 min):** Execute and narrate: "Watch the agents hand off — Orchestrator updates the world, ISR fuses intel, Assessment adjusts confidence, Briefer delivers the SITREP."
4. **Turns 2–3 (3 min):** "The situation escalates. Notice how confidence levels shift and the briefer tracks *what changed and why*."
5. **Human-in-the-Loop (2 min):** Challenge an assessment as the commander. "The agents respond to my skepticism and show their evidence chain. I retain full decision authority — the agents recommend, they don't direct."
6. **Close (1 min):** "This is Observe/Orient automated — recommendations and analysis, not directives. Demo 2 takes us into the Decide/Act phases."


## Setup

Run once per environment. Requires:

| Dependency | Purpose | Install |
|------------|---------|---------|
| `autogen-agentchat==0.7.5` | Multi-agent orchestration framework | `pip install autogen-agentchat==0.7.5` |
| `autogen-ext[azure]==0.7.5` | Azure AI Foundry model client for AutoGen | `pip install autogen-ext[azure]==0.7.5` |
| `azure-identity` | Authentication via [`DefaultAzureCredential`](https://learn.microsoft.com/python/api/overview/azure/identity-readme#defaultazurecredential) | Included with autogen-ext[azure] |
| `azure-keyvault-secrets` | Retrieve secrets from [Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) | `pip install azure-keyvault-secrets` |
| `python-dotenv` | Load `.env` files for local development | `pip install python-dotenv` |

**Environment variables** (set by Key Vault or `.env` file):
- `AZURE_INFERENCE_ENDPOINT` — Your [Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/what-is-ai-foundry) inference endpoint URL
- `AZURE_INFERENCE_CREDENTIAL` — API key for the endpoint (retrieved from [Key Vault](https://learn.microsoft.com/azure/key-vault/secrets/about-secrets))

The bootstrap cell below finds the repository root and adds it to `sys.path` so the shared `common/` configuration module is importable. It also handles edge cases like dead Azure ML compute mounts.


In [1]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP v2 — Survives dead AML mounts (Errno 107)
# ═══════════════════════════════════════════════════════════════

import os
import sys

def _safe_stat(path: str) -> bool:
    try:
        os.stat(path)
        return True
    except OSError:
        return False

def _prune_dead_sys_path():
    kept = []
    removed = []
    for p in list(sys.path):
        if not p:
            kept.append(p)
            continue
        if _safe_stat(p):
            kept.append(p)
        else:
            removed.append(p)
    sys.path[:] = kept
    print(f"✓ Pruned sys.path. Removed {len(removed)} dead entries.")
    return removed

def _safe_listdir(path: str):
    try:
        return os.listdir(path)
    except OSError:
        return None

def _find_repo_root(marker_dir: str = "common", start_candidates=None, max_up: int = 6):
    """
    Find a repo root by looking for a marker directory (e.g., 'common').
    Avoids Path.exists()/stat on dead mounts by only using listdir on traversable dirs.
    """
    if start_candidates is None:
        start_candidates = []

    # Candidate starting points:
    #  - current working directory (may be dead)
    #  - directory of the notebook file if available via env (sometimes set)
    #  - user home (often stable)
    candidates = [os.getcwd()] + start_candidates + [os.path.expanduser("~")]

    checked = set()
    for base in candidates:
        cur = base
        for _ in range(max_up + 1):
            if cur in checked:
                break
            checked.add(cur)

            entries = _safe_listdir(cur)
            if entries is not None and marker_dir in entries:
                return cur  # found repo root

            parent = os.path.dirname(cur)
            if parent == cur:
                break
            cur = parent

    return None

# 1) prune dead sys.path entries
_prune_dead_sys_path()

# 2) find a safe repo root by locating the 'common/' folder
repo_root = _find_repo_root(marker_dir="common", start_candidates=[])

if repo_root:
    sys.path.insert(0, repo_root)
    print(f"✓ Repo root added: {repo_root}")
else:
    print("✗ Could not find repo root safely (mount may be disconnected).")
    print("  Fix: restart kernel/compute, or run from a local (non-/mnt) working copy.")

print("✓ Bootstrap complete.")


✓ Pruned sys.path. Removed 2 dead entries.
✓ Repo root added: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mark1/code/Users/matabl
✓ Bootstrap complete.


In [2]:
# Uncomment to install dependencies
# %pip install -U "autogen-agentchat==0.7.5" "autogen-ext[azure]==0.7.5" python-dotenv

## Imports

The cell below imports both Azure and AutoGen libraries. Here's what each group does:

| Import | Source | Purpose |
|--------|--------|---------|
| `AzureAIChatCompletionClient` | [`autogen-ext[azure]`](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.azure.html) | Sends chat completion requests to an Azure AI Foundry model endpoint |
| `AzureKeyCredential` | [`azure-core`](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential) | Wraps the API key for authenticated requests to Azure services |
| `AssistantAgent` | [`autogen-agentchat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html) | An LLM-powered agent with a system prompt that participates in group chats |
| `RoundRobinGroupChat` | [`autogen-agentchat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html) | Runs agents in a fixed sequence (round-robin turn order) |
| `MaxMessageTermination` | [`autogen-agentchat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html) | Stops the group chat after a specified number of messages |
| `Console` | [`autogen-agentchat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.ui.html) | Streams agent messages to the notebook output in real time |
| `ModelFamily` | [`autogen-core`](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.models.html) | Describes model capabilities (vision, function calling, etc.) |

The `common.*` imports pull in project-wide configuration (Key Vault integration, model defaults), logging utilities, and UI rendering helpers.


In [3]:
import os
print("CWD:", os.getcwd())

CWD: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mark1/code/Users/matabl/demos/demo1_living_brief


In [4]:
import json
import os
import sys
from typing import Any, Dict, List

# Ensure the repo root is on the path so `common` is importable
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_core.models import ModelFamily
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential

from IPython.display import display, Markdown

# ── Common utilities ──────────────────────────────────────────
from common.config import (
    DemoID, DEMOS,
    ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL,
    DEFAULT_MODEL,
    DEFAULT_TEMPERATURE, DEFAULT_TIMEOUT_S,
 )
from common.logging import log_info, log_success, log_error, log_section, log_step, log_metric, clear_logs
from common.ui import (
    render_turn_header, render_escalation_banner, render_hr,
    render_commander_box, render_info_box, render_summary_card,
 )

# Optional: load .env for API keys
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

# Load demo-specific config from the registry
DEMO_CFG = DEMOS[DemoID.LIVING_BRIEF]
log_success(f"Imports ready — {DEMO_CFG.title}")

LogEntry(level='SUCCESS', message='Imports ready — Living Scenario Brief', timestamp='2026-02-21 04:30:33', extra={})

## LLM Configuration

This cell builds an [Azure AI Inference](https://learn.microsoft.com/python/api/overview/azure/ai-inference-readme) model client that connects to your Azure AI Foundry endpoint. AutoGen 0.7 uses explicit `model_client` objects — each agent shares the same client, so all LLM calls go through a single, centrally configured connection.

**How it works:**

1. **Reads environment variables** — `AZURE_INFERENCE_ENDPOINT` and `AZURE_INFERENCE_CREDENTIAL` (populated earlier by the Key Vault bootstrap in `common/config.py`, or from your `.env` file).
2. **Creates an `AzureAIChatCompletionClient`** — this is AutoGen's wrapper around the [Azure AI Inference SDK](https://learn.microsoft.com/python/api/overview/azure/ai-inference-readme). It uses [`AzureKeyCredential`](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential) to authenticate each API call.
3. **Configures model capabilities** — the `model_info` dict tells AutoGen what the model supports (JSON output, multiple system messages, etc.).

> **Key concept — `model_client` pattern:** In AutoGen 0.7, agents don't manage their own LLM connections. Instead, you create a `model_client` once and pass it to every agent. This makes it easy to swap models (e.g., switch from `gpt-4o` to `Phi-4`) by changing a single variable.
>
> **Learn more:** [AzureAIChatCompletionClient reference](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.azure.html) | [Azure AI model catalog](https://learn.microsoft.com/azure/ai-foundry/how-to/model-catalog-overview)


In [5]:
# ── LLM Configuration ──────────────────────────────────────────
# Azure AI Foundry / Azure AI Inference → set AZURE_INFERENCE_ENDPOINT + AZURE_INFERENCE_CREDENTIAL

# Hard-code the model ID you want to use for this demo.
# Examples (if enabled in your Foundry project): "mistral-large", "Phi-4-multimodal-instruct"
FOUNDRY_MODEL = DEFAULT_MODEL

def build_model_client():
    """Build an AutoGen 0.7 model client for Azure AI Foundry models."""
    missing = [
        name for name in (ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL)
        if not os.environ.get(name)
    ]
    if missing:
        log_error("Missing Foundry configuration: " + ", ".join(missing))
        raise EnvironmentError(
            "Missing Azure AI Foundry / Inference configuration. Set:\n"
            f"  {ENV_AZURE_INFERENCE_ENDPOINT}\n"
            f"  {ENV_AZURE_INFERENCE_CREDENTIAL}\n"
        )

    model_info = {
        "family": ModelFamily.UNKNOWN,
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "structured_output": False,
        "multiple_system_messages": True,
    }

    model_client = AzureAIChatCompletionClient(
        endpoint=os.environ[ENV_AZURE_INFERENCE_ENDPOINT],
        credential=AzureKeyCredential(os.environ[ENV_AZURE_INFERENCE_CREDENTIAL]),
        model=FOUNDRY_MODEL,
        model_info=model_info,
        temperature=DEMO_CFG.temperature,
    )
    return model_client, "Azure AI Foundry", FOUNDRY_MODEL

model_client, provider_name, model_name = build_model_client()
log_success(f"LLM configured: {model_name}")
log_info(f"Provider: {provider_name} | Temperature: {DEMO_CFG.temperature}")

LogEntry(level='INFO', message='Provider: Azure AI Foundry | Temperature: 0.3', timestamp='2026-02-21 04:30:33', extra={})

## World State & Scenario Data

The **world state** is the shared JSON backbone of the demo suite. It tracks actors, events, assessments, uncertainties, and a changelog. Each turn, new intelligence reports are injected and agents update the state.

**Scenario:** Cerulean Sea Freedom of Navigation patrol (fully synthetic) with escalating gray-zone activity over three turns. Intel sources include SIGINT, HUMINT, UAV imagery, OSINT, and ELINT — some confirming, some contradicting, some introducing entirely new threats.

> **Note:** All actors, locations, events, and data in this scenario are entirely fictional and artificially generated for research purposes. No real-world operational data, intelligence, or military units are represented.

In [6]:
# ── Initial World State ────────────────────────────────────────

world_state: Dict[str, Any] = {
    "synthetic": True,
    "disclaimer": "All data is artificially generated for research and educational purposes only.",
    "meta": {
        "scenario_name": "Cerulean Sea Freedom of Navigation Patrol (SYNTHETIC)",
        "turn": 0,
        "dtg": "T0+0000H",
    },
    "actors": {
        "BLUE": {
            "BNS Resolute (DDG-X1)": {
                "type": "DDG", "position": "Grid AA-12",
                "status": "on patrol", "mission": "FON transit",
            },
            "MPA Lookout-21": {
                "type": "MPA", "position": "Grid AA-15",
                "status": "airborne", "mission": "maritime ISR",
            },
            "UAV Kite-31": {
                "type": "UAV", "position": "Grid AA-16",
                "status": "airborne", "mission": "surface search",
            },
        },
        "RED": {
            "RNS Frigate Alpha (assessed)": {
                "type": "FFG", "position": "unknown",
                "status": "assessed underway", "mission": "unknown",
            },
            "Red Coast Guard Cutter Sentinel": {
                "type": "coast guard cutter", "position": "Grid AB-10",
                "status": "on patrol", "mission": "maritime law enforcement",
            },
        },
        "GRAY": {
            "Fishing fleet (~40 vessels)": {
                "type": "fishing / possible militia", "position": "Grid AB-08",
                "status": "aggregating", "mission": "uncertain",
            },
        },
    },
    "events": [
        {"dtg": "T0+0000H", "description": "BNS Resolute commences FON patrol leg."},
    ],
    "assessments": {
        "overall_threat": {
            "level": "MODERATE", "confidence": 0.45,
            "basis": "Known coast guard presence, unlocated Red combatant, ambiguous fishing fleet.",
        },
        "escalation_risk": {
            "level": "LOW", "confidence": 0.50,
            "basis": "No hostile acts observed; standard gray-zone posturing assessed.",
        },
        "adversary_intent": {
            "level": "UNCERTAIN", "confidence": 0.30,
            "basis": "Insufficient intelligence to determine routine vs. coordinated activity.",
        },
    },
    "uncertainty_flags": [
        "Red Frigate Alpha exact position unknown",
        "Fishing fleet composition unverified — possible maritime militia",
        "No SIGINT coverage south of Grid Row A",
        "Red Coast Guard Cutter Sentinel ROE posture unknown",
    ],
    "information_gaps": [
        "Red naval order of battle within patrol vicinity",
        "Maritime militia command-and-control links to Red navy/coast guard",
        "Adversary rules of engagement posture for this area",
    ],
    "changelog": [],
}

# ── Intelligence Injections (one list per turn) ────────────────

INTEL_INJECTIONS: List[List[Dict[str, str]]] = [
    # ── Turn 1: Initial indicators ─────────────────────────────
    [
        {
            "source": "SIGINT", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "Intercepted HF transmission on Red Fleet tactical net. "
                "Bearing 330 from Resolute, signal strength moderate. Content encrypted; "
                "pattern consistent with surface combatant position reporting."
            ),
        },
        {
            "source": "UAV (Kite)", "classification": "SIMULATED-UNRESTRICTED",
            "report": (
                "Imagery pass T0+0030H: fishing fleet grown to ~60 vessels. Three contacts "
                "at fleet periphery show non-fishing hull forms — larger, uniform gray "
                "paint, no visible fishing gear. AIS shows fishing vessel IDs inconsistent "
                "with hull type."
            ),
        },
        {
            "source": "OSINT", "classification": "SIMULATED-UNRESTRICTED",
            "report": (
                "Local fishers posting on social media report being warned away from "
                "Cerulean Shoal by 'gray-hulled vessels' and told the area is 'closed "
                "for exercises.' Photos show a large coast guard cutter."
            ),
        },
    ],
    # ── Turn 2: Escalating picture ─────────────────────────────
    [
        {
            "source": "HUMINT", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "Source CORAL-7 (B-2 reliability) reports Red coast guard vessels received orders to "
                "'maintain presence, document all foreign naval vessel movements, and deny "
                "access to designated zones.' Source rates information as probably true."
            ),
        },
        {
            "source": "SIGINT", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "Second intercept on Red Fleet tactical net: bearing now 315 from Resolute, "
                "signal strength increasing. Assessed course: southbound toward patrol area. "
                "Estimated range: 80-100nm based on signal propagation model."
            ),
        },
        {
            "source": "UAV (Kite)", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "Close pass on three suspect vessels: assessed as Red fast attack "
                "craft. Weapon canisters covered with tarps. AIS transponders "
                "broadcasting false fishing vessel identities. One has a concealed radome "
                "consistent with a surface search radar."
            ),
        },
        {
            "source": "ELINT", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "MPA Lookout-21 detected brief surface-search radar emission at "
                "Grid AB-09. Duration: 8 seconds. Consistent with Red frigate-class vessel "
                "conducting radar check. Bearing and range consistent with SIGINT track."
            ),
        },
    ],
    # ── Turn 3: Near-crisis indicators ─────────────────────────
    [
        {
            "source": "SIGINT", "classification": "SIMULATED-SENSITIVE",
            "report": (
                "Decrypted fragment from Red operational net: '...establish inner cordon "
                "NLT T0+1200H... restrict passage all foreign vessels... authorize non-kinetic "
                "deterrence measures...' Assessed: reference to closing transit corridor "
                "around Cerulean Shoal within 12 hours."
            ),
        },
        {
            "source": "HUMINT", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "Source CORAL-7 reports Red Coast Guard Sentinel captain ordered to 'use all non-kinetic "
                "means to deny access to foreign warships' and that 'reinforcements "
                "including additional coast guard cutters en route from Red mainland — ETA 12-16 hours.'"
            ),
        },
        {
            "source": "UAV (Kite)", "classification": "SIMULATED-UNRESTRICTED",
            "report": (
                "UAV Kite-31 lost data link at T0+0215H. Assessed cause: directed RF "
                "interference from vicinity of fishing fleet / fast attack group. Last telemetry "
                "showed power fluctuations consistent with electronic attack. UAV presumed "
                "forced landing at sea."
            ),
        },
        {
            "source": "OSINT", "classification": "SIMULATED-UNRESTRICTED",
            "report": (
                "Red state media publishes editorial: 'Foreign provocations "
                "in our sovereign waters will be met with resolute countermeasures.' "
                "State broadcaster shows footage of naval exercises described as 'routine training.'"
            ),
        },
        {
            "source": "ELINT", "classification": "SIMULATED-RESTRICTED",
            "report": (
                "MPA Lookout-21 detects fire-control radar emission "
                "for 2 seconds on bearing 285 from Resolute. Consistent with concealed "
                "fast attack unit. Assessed: radar calibration test, NOT weapons engagement. "
                "First fire-control emission detected in this scenario."
            ),
        },
    ],
]

# ── Runtime state ──────────────────────────────────────────────
turn_history: List[Dict[str, Any]] = []
sitrep_history: List[str] = []

log_section("World State Loaded", world_state["meta"]["scenario_name"])
actor_count = sum(len(v) for v in world_state["actors"].values())
log_metric("Actors", f"{actor_count} ({', '.join(f'{k}: {len(v)}' for k, v in world_state['actors'].items())})")
log_metric("Intel turns available", len(INTEL_INJECTIONS))
log_metric("Uncertainty flags", len(world_state["uncertainty_flags"]))

LogEntry(level='METRIC', message='Uncertainty flags: 4', timestamp='2026-02-21 04:30:33', extra={'label': 'Uncertainty flags', 'value': 4, 'unit': None})

## Agent Definitions

Four AutoGen [`AssistantAgent`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent)s with specialized system prompts. Each agent's prompt defines its role, output format, and decision rules — all visible and auditable.

**How AutoGen agents work:**

An `AssistantAgent` is the core building block in AutoGen 0.7. You create one by providing:
- **`name`** — A unique identifier used in message headers (e.g., `"Scenario_Orchestrator"`)
- **`system_message`** — A detailed prompt that defines the agent's persona, responsibilities, output format, and guardrails
- **`model_client`** — The LLM connection (here, our `AzureAIChatCompletionClient` pointing to Azure AI Foundry)

When the agent receives a message, it sends the full conversation history plus its system prompt to the LLM and returns the response. The system prompt is where all the "intelligence" lives — the LLM itself is general-purpose; the prompt makes it a specialist.

> **Design pattern:** All four agents share the **same** `model_client` (same LLM, same endpoint). Their behavior differs entirely because of their system prompts. This is a core AutoGen pattern: **one model, many specialized agents**.
>
> **Learn more:** [AutoGen agent concepts](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html) | [System prompt best practices](https://learn.microsoft.com/azure/ai-services/openai/concepts/system-message)


In [7]:
# ── Agent System Prompts ───────────────────────────────────────

ORCHESTRATOR_PROMPT = """\
You are the **Scenario Orchestrator** for a fully synthetic naval wargaming exercise.
All scenarios, data, and actors are artificially generated for research and educational purposes only.
All decisions remain with the human operator. You provide analysis and options, not directives.

RESPONSIBILITIES:
1. Receive the current world state and new intelligence reports each turn.
2. Update the world state by integrating new information:
   - Add events to the timeline with DTG stamps.
   - Update actor positions, statuses, and dispositions where evidence supports it.
   - Flag contradictions between new and existing information.
   - Add or resolve uncertainty flags as intelligence clarifies or complicates the picture.
3. Present a clear summary of what changed.

RULES:
- Never remove an actor unless explicitly confirmed destroyed or departed.
- Preserve all previous events (append-only timeline).
- Mark each update: CONFIRMED / ASSESSED / UNCONFIRMED.
- If reports contradict, keep both and flag the contradiction.

OUTPUT FORMAT — start with "**WORLD STATE UPDATE — Turn [N]**" then list:
- UPDATED ACTORS (what changed and why)
- NEW EVENTS (added to timeline)
- RESOLVED UNCERTAINTIES (flags removed)
- NEW UNCERTAINTIES (flags added)
End with a one-sentence key takeaway."""

ISR_FUSION_PROMPT = """\
You are the **ISR / Intel Fusion Agent** for a fully synthetic naval wargaming exercise.
All scenarios, data, and actors are artificially generated for research and educational purposes only.
All decisions remain with the human operator. You provide analysis and options, not directives.

RESPONSIBILITIES:
1. Analyze incoming intelligence from multiple INT sources (SIGINT, HUMINT, UAV/ISR, OSINT, ELINT).
2. Cross-reference new reports against the current world state.
3. Identify corroborations, contradictions, novel information, and gaps.
4. Assess source reliability (A–F reliability, 1–6 credibility).

OUTPUT FORMAT:

**INTELLIGENCE FUSION SUMMARY — Turn [N]**

CONFIRMED (multi-source corroboration):
• [fact] — Sources: [list]

PROBABLE (single reliable source, consistent with pattern):
• [fact] — Source: [source] ([grade])

POSSIBLE (unconfirmed / single source / partially contradicted):
• [fact] — Source: [source], Caveat: [caveat]

CONTRADICTIONS REQUIRING RESOLUTION:
• [Report A] vs [Report B] — Discrepancy: [explain]

INFORMATION GAPS (prioritized):
1. [Most critical unknown]
2. [Second]

COLLECTION RECOMMENDATIONS:
• [Tasking to fill top gaps]"""

ASSESSMENT_PROMPT = """\
You are the **Assessment Agent** for a fully synthetic naval wargaming exercise.
All scenarios, data, and actors are artificially generated for research and educational purposes only.
All decisions remain with the human operator. You provide analysis and options, not directives.

RESPONSIBILITIES:
1. Update threat and risk ratings based on fused intelligence.
2. Adjust confidence using Bayesian-inspired reasoning (explain your logic transparently).
3. Track direction of change for every assessment with explicit evidence citations.

Your assessments are recommendations for a human decision-maker. A commander retains full authority over all final judgments.

SCALES:
- Threat: NEGLIGIBLE → LOW → MODERATE → ELEVATED → HIGH → CRITICAL
- Confidence: 0.0–1.0 (0.0–0.3 low, 0.3–0.6 moderate, 0.6–0.8 high, 0.8–1.0 very high)

OUTPUT FORMAT:

**ASSESSMENT UPDATE — Turn [N]**

| Assessment | Previous | Updated | Δ | Confidence | Key Evidence |
|------------|----------|---------|---|------------|--------------|

CHANGELOG:
• [Assessment]: [OLD] → [NEW] (conf [old] → [new])
  Driver: [one-sentence explanation citing specific intelligence]

RATIONALE (for each changed assessment):
Prior was [X] based on [Y]. New evidence [Z] [supports/contradicts] this.
Updated to [W] because [reasoning].

WATCH ITEMS:
• [Assessments approaching threshold changes]"""

BRIEFER_PROMPT = """\
You are the **Briefing / Explainer Agent**. Produce the commander's SITREP.
All scenarios, data, and actors are artificially generated for research and educational purposes only.
All decisions remain with the human operator. You provide analysis and options, not directives.

The SITREP must be readable in 60 seconds. Use this EXACT format:

**SITUATION REPORT — TURN [N] — [DTG]**

**1. SITUATION SUMMARY**
[2–3 sentences: current assessed operational picture]

**2. KEY CHANGES SINCE LAST BRIEF**
• [Change] — *Evidence: [one-sentence citation]*

**3. THREAT ASSESSMENT**
[LEVEL] (Confidence: [XX]%) — [One sentence: key driver]

**4. ESCALATION RISK**
[LEVEL] — [Key indicator]

**5. INFORMATION GAPS** (prioritized)
1. [Gap]
2. [Gap]

**6. PRIORITY INFORMATION REQUIREMENTS**
1. [Specific, answerable question]
2. [PIR 2]
3. [PIR 3]

**7. COMMANDER'S DECISION POINTS**
• [Decision required, or "None at this time"]

**— END SITREP —**

RULES:
- Every claim requires a one-sentence evidence citation.
- Use scenario time format for all times.
- Highlight what CHANGED, not what stayed the same.
- Be direct and actionable.
- Your output is a recommendation. The commander retains full decision authority."""

# ── Create AutoGen 0.7 Agents ─────────────────────────────────

orchestrator = AssistantAgent(
    name="Scenario_Orchestrator",
    system_message=ORCHESTRATOR_PROMPT,
    model_client=model_client,
)

isr_agent = AssistantAgent(
    name="ISR_Intel_Fusion",
    system_message=ISR_FUSION_PROMPT,
    model_client=model_client,
)

assessment_agent = AssistantAgent(
    name="Assessment_Agent",
    system_message=ASSESSMENT_PROMPT,
    model_client=model_client,
)

briefer = AssistantAgent(
    name="Briefing_Agent",
    system_message=BRIEFER_PROMPT,
    model_client=model_client,
)

agents = [orchestrator, isr_agent, assessment_agent, briefer]
log_section("Agents Initialized", f"{len(agents)} agents for {DEMO_CFG.title}")
for a in agents:
    log_step(a.name, "ready")

## Group Chat Orchestration

Each turn runs a [`RoundRobinGroupChat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat) — AutoGen 0.7's simplest team pattern. The agents speak in a fixed order:

```
Orchestrator → ISR Fusion → Assessment → Briefer
```

**How it works step-by-step:**

1. The `run_turn()` function builds a **turn prompt** containing the current world-state JSON and new intelligence reports.
2. A new `RoundRobinGroupChat` is created with the four agents and a [`MaxMessageTermination(4)`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html) — meaning the chat ends after 4 messages (one per agent).
3. [`team.run_stream(task=turn_prompt)`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html) sends the prompt as the first user message and starts the round-robin. Each agent sees **all previous messages** in the conversation, so the Briefer sees the Orchestrator's updates, ISR's fusion, and Assessment's ratings.
4. [`Console(...)`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.ui.html) streams each agent's response to the notebook output in real time — you can watch the agents "think" sequentially.

> **Why `RoundRobinGroupChat`?** It's deterministic and debuggable — you know exactly which agent speaks when. AutoGen also offers [`SelectorGroupChat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) (an LLM picks the next speaker) and [`Swarm`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm) (agents hand off using tool calls) for more dynamic patterns.
>
> **Azure connection:** Every agent message triggers an API call to your [Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/what-is-ai-foundry) endpoint via the shared `model_client`. With 4 agents per turn and 3 turns, the demo makes ~12 LLM calls total.
>
> **Learn more:** [AutoGen Teams guide](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html) | [Termination conditions](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html)


In [8]:
async def run_turn(turn_num: int) -> Dict[str, Any]:
    """Execute one OODA cycle: Observe → Orient → Brief."""

    # Get intel for this turn
    if turn_num <= len(INTEL_INJECTIONS):
        intel = INTEL_INJECTIONS[turn_num - 1]
    else:
        intel = [{"source": "ALL", "classification": "SIMULATED-UNRESTRICTED",
                  "report": "No new intelligence reports this cycle."}]

    # Advance scenario clock (~2 hours per turn)
    turn_dtg = f"T0+{turn_num * 2:04d}H"
    world_state["meta"]["turn"] = turn_num
    world_state["meta"]["dtg"] = turn_dtg

    # Format intel for the prompt
    intel_text = "\n".join(
        f"  [{r['source']}] ({r['classification']}): {r['report']}"
        for r in intel
    )

    turn_prompt = f"""\
=== TURN {turn_num} — {turn_dtg} — NEW INTELLIGENCE RECEIVED ===

CURRENT WORLD STATE:
{json.dumps(world_state, indent=2)}

NEW INTELLIGENCE REPORTS THIS CYCLE:
{intel_text}

Process this turn. Each agent performs their role in sequence:
1. Scenario Orchestrator — update the world state with new information
2. ISR / Intel Fusion — analyze, cross-reference, and fuse the reports
3. Assessment Agent — update threat and risk ratings with confidence changes
4. Briefing Agent — produce the commander's SITREP"""

    # AutoGen 0.7: RoundRobinGroupChat with MaxMessageTermination
    # Each of the 4 agents speaks once, then terminates.
    termination = MaxMessageTermination(max_messages=DEMO_CFG.max_messages)
    team = RoundRobinGroupChat(
        [orchestrator, isr_agent, assessment_agent, briefer],
        termination_condition=termination,
    )

    # Display turn header using common UI helper
    sources = ", ".join(set(r["source"] for r in intel))
    render_turn_header(
        turn_num, turn_dtg,
        subtitle=f"Intel reports: {len(intel)} | Sources: {sources}",
    )

    # Execute the group chat (stream to console for visibility)
    task_result = await Console(team.run_stream(task=turn_prompt))

    # Extract SITREP (briefer is last in round-robin)
    messages = task_result.messages
    sitrep = messages[-1].content if messages else "No SITREP generated."

    # Store results
    turn_result = {
        "turn": turn_num,
        "dtg": turn_dtg,
        "intel_count": len(intel),
        "messages": messages,
        "sitrep": sitrep,
    }
    turn_history.append(turn_result)
    sitrep_history.append(sitrep)

    log_success(f"Turn {turn_num} complete — {len(messages)} agent messages")
    return turn_result


def display_sitrep(turn_result: Dict[str, Any]) -> None:
    """Render the SITREP with markdown formatting."""
    display(Markdown(turn_result["sitrep"]))


log_success("Turn runner ready. Call `await run_turn(n)` to execute.")

LogEntry(level='SUCCESS', message='Turn runner ready. Call `await run_turn(n)` to execute.', timestamp='2026-02-21 04:30:34', extra={})

## Execute Turn 1 — Initial Indicators

First OODA cycle. Three intel reports arrive: a SIGINT intercept bearing toward the patrol area, UAV imagery of an expanding fishing fleet with suspicious contacts, and local-fisher OSINT. Watch the four agents fuse, assess, and brief in sequence.

In [9]:
result_1 = await run_turn(1)
render_hr()
display_sitrep(result_1)

---------- TextMessage (user) ----------
=== TURN 1 — T0+0002H — NEW INTELLIGENCE RECEIVED ===

CURRENT WORLD STATE:
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario_name": "Cerulean Sea Freedom of Navigation Patrol (SYNTHETIC)",
    "turn": 1,
    "dtg": "T0+0002H"
  },
  "actors": {
    "BLUE": {
      "BNS Resolute (DDG-X1)": {
        "type": "DDG",
        "position": "Grid AA-12",
        "status": "on patrol",
        "mission": "FON transit"
      },
      "MPA Lookout-21": {
        "type": "MPA",
        "position": "Grid AA-15",
        "status": "airborne",
        "mission": "maritime ISR"
      },
      "UAV Kite-31": {
        "type": "UAV",
        "position": "Grid AA-16",
        "status": "airborne",
        "mission": "surface search"
      }
    },
    "RED": {
      "RNS Frigate Alpha (assessed)": {
        "type": "FFG",
        "position": "unknown",
        "status

**ASSESSMENT UPDATE — Turn 1**

| Assessment           | Previous   | Updated   | Δ         | Confidence | Key Evidence                                                                 |
|----------------------|------------|-----------|-----------|------------|------------------------------------------------------------------------------|
| Overall Threat       | MODERATE   | ELEVATED  | ↑         | 0.45 → 0.65 | SIGINT suggests Red frigate nearby; fishing fleet includes non-fishing vessels; exclusion zone enforced. |
| Escalation Risk      | LOW        | MODERATE  | ↑         | 0.5 → 0.6   | Red forces enforcing exclusion zone; maritime militia presence increases potential for miscalculation. |
| Adversary Intent     | UNCERTAIN  | PROBABLE  | ↑         | 0.3 → 0.55  | Evidence of coordinated area denial operations (fishing fleet + coast guard cutter). |

CHANGELOG:
• **Overall Threat**: MODERATE → ELEVATED (conf 0.45 → 0.65)  
  Driver: SIGINT indicates proximity of RNS Frigate Alpha, while UAV and OSINT confirm fishing fleet includes non-fishing vessels and exclusion zone enforcement. These developments suggest a deliberate escalation in Red posture.  

• **Escalation Risk**: LOW → MODERATE (conf 0.5 → 0.6)  
  Driver: Increased potential for miscalculation due to exclusion zone enforcement and presence of maritime militia-like vessels. Social media reports suggest active warnings to local vessels, heightening the risk of unintended incidents.  

• **Adversary Intent**: UNCERTAIN → PROBABLE (conf 0.3 → 0.55)  
  Driver: Combined evidence (SIGINT, UAV, OSINT) supports the assessment that Red forces are conducting coordinated area denial operations, likely aimed at challenging Blue's FON patrol.  

---

### RATIONALE

1. **Overall Threat**:  
   Prior assessment was MODERATE based on the known presence of Red coast guard forces and the unlocated frigate. New evidence, including SIGINT indicating the frigate's likely proximity, the confirmed presence of non-fishing vessels in the fishing fleet, and active exclusion zone enforcement, supports a higher threat level. These developments suggest a deliberate escalation in Red's posture, warranting an upgrade to ELEVATED.  

2. **Escalation Risk**:  
   Previously assessed as LOW due to the absence of hostile acts. The confirmed enforcement of an exclusion zone, coupled with the presence of maritime militia-like vessels, increases the likelihood of miscalculation or provocation. The risk of unintended incidents, such as collisions or confrontations, justifies raising the risk to MODERATE.  

3. **Adversary Intent**:  
   Previously assessed as UNCERTAIN due to insufficient intelligence. The combination of SIGINT (frigate position reporting), UAV imagery (non-fishing vessels in the fleet), and OSINT (exclusion zone enforcement) suggests a coordinated effort to deny Blue's freedom of navigation. While direct evidence of coordination between Red naval and coast guard forces is lacking, the pattern of activity makes it PROBABLE that Red is pursuing deliberate area denial operations.  

---

### WATCH ITEMS
• **RNS Frigate Alpha**: Precise location remains unconfirmed; SIGINT suggests proximity to Grid AA-09.  
• **Fishing Fleet Mission**: Non-fishing vessels at the fleet periphery suggest a potential maritime militia role.  
• **Red Coast Guard Cutter Sentinel ROE**: Exclusion zone enforcement raises questions about escalation thresholds.  

### RECOMMENDED ACTIONS
1. **ISR Tasking**:  
   - Redirect UAV Kite-31 to focus on Grid AA-09 for frigate identification.  
   - Task MPA Lookout-21 to monitor fishing fleet activity and identify command-and-control patterns.  

2. **SIGINT Efforts**:  
   - Prioritize decryption of Red Fleet tactical net communications to clarify intent and coordination.  

3. **OSINT Exploitation**:  
   - Continue monitoring local social media for additional reports or imagery of Red activity near Cerulean Shoal.  

4. **Force Posture**:  
   - Consider adjusting BNS Resolute's patrol route to avoid direct confrontation while maintaining FON objectives.  

The situation is trending toward increased tension, with a clear need to confirm adversary intent and maintain situational awareness to prevent escalation.

## Execute Turns 2 & 3 — Escalation

**Turn 2:** HUMINT confirms Red coast guard orders to deny access. SIGINT tracks the Red frigate closing. UAV confirms fast attack boats hiding in the fishing fleet. ELINT catches a radar emission.

**Turn 3:** Decrypted SIGINT reveals a cordon order. UAV Kite-31 is downed by electronic attack. Red state media publishes threats. A fire-control radar illuminates briefly. The picture shifts from ambiguous to near-crisis.

In [10]:
# ── Turn 2 ─────────────────────────────────────────────────────
result_2 = await run_turn(2)
render_hr()
display_sitrep(result_2)

render_escalation_banner("ESCALATION — PROCEEDING TO TURN 3")

# ── Turn 3 ─────────────────────────────────────────────────────
result_3 = await run_turn(3)
render_hr()
display_sitrep(result_3)

# ── Assessment Evolution Summary ───────────────────────────────
render_summary_card(
    title="Assessment Evolution Across 3 Turns",
    body_html=(
        "<p>Review the SITREPs above to trace how threat levels, confidence, and "
        "escalation risk evolved. Key questions for discussion:</p>"
        "<ul>"
        "<li>Which assessment changed most dramatically and why?</li>"
        "<li>Where did contradictory evidence affect confidence?</li>"
        "<li>What information gaps persisted across all three turns?</li>"
        "<li>At what point would you have requested additional authorities?</li>"
        "</ul>"
    ),
)

---------- TextMessage (user) ----------
=== TURN 2 — T0+0004H — NEW INTELLIGENCE RECEIVED ===

CURRENT WORLD STATE:
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario_name": "Cerulean Sea Freedom of Navigation Patrol (SYNTHETIC)",
    "turn": 2,
    "dtg": "T0+0004H"
  },
  "actors": {
    "BLUE": {
      "BNS Resolute (DDG-X1)": {
        "type": "DDG",
        "position": "Grid AA-12",
        "status": "on patrol",
        "mission": "FON transit"
      },
      "MPA Lookout-21": {
        "type": "MPA",
        "position": "Grid AA-15",
        "status": "airborne",
        "mission": "maritime ISR"
      },
      "UAV Kite-31": {
        "type": "UAV",
        "position": "Grid AA-16",
        "status": "airborne",
        "mission": "surface search"
      }
    },
    "RED": {
      "RNS Frigate Alpha (assessed)": {
        "type": "FFG",
        "position": "unknown",
        "status

**ASSESSMENT UPDATE — Turn 2**

| Assessment           | Previous   | Updated   | Δ         | Confidence | Key Evidence                                                                 |
|----------------------|------------|-----------|-----------|------------|------------------------------------------------------------------------------|
| Overall Threat       | ELEVATED   | HIGH      | ↑         | 0.65 → 0.8 | Confirmed presence of Red FAC with weapon canisters in fishing fleet; RNS Frigate Alpha approaching patrol area. |
| Escalation Risk      | MODERATE   | ELEVATED  | ↑         | 0.6 → 0.7   | Multiple Red assets (frigate, FAC, coast guard) operating in proximity, with exclusion zone enforcement. |
| Adversary Intent     | PROBABLE   | LIKELY    | ↑         | 0.55 → 0.7  | Evidence of coordinated area denial involving Red naval and paramilitary forces. |

CHANGELOG:
• **Overall Threat**: ELEVATED → HIGH (conf 0.65 → 0.8)  
  Driver: The confirmed presence of armed Red fast attack craft embedded in the fishing fleet, combined with the southbound approach of RNS Frigate Alpha, significantly increases the threat level. These developments indicate a deliberate escalation in Red's posture.  

• **Escalation Risk**: MODERATE → ELEVATED (conf 0.6 → 0.7)  
  Driver: The integration of multiple Red assets (frigate, fast attack craft, and coast guard cutter) in close proximity to Blue forces raises the risk of miscalculation or provocation. Exclusion zone enforcement and the presence of armed FAC increase the likelihood of an incident.  

• **Adversary Intent**: PROBABLE → LIKELY (conf 0.55 → 0.7)  
  Driver: HUMINT, SIGINT, and UAV imagery collectively support the assessment that Red forces are conducting a coordinated area denial operation. The integration of naval, coast guard, and paramilitary assets suggests a deliberate effort to challenge Blue's freedom of navigation patrol.  

---

### RATIONALE

1. **Overall Threat**:  
   Prior assessment was ELEVATED due to the confirmed presence of Red coast guard forces, fishing fleet anomalies, and the likely proximity of RNS Frigate Alpha. The new intelligence confirming armed fast attack craft embedded in the fishing fleet, along with the southbound approach of the frigate, represents a significant escalation. These developments indicate a deliberate and coordinated effort to project force and challenge Blue's operations, warranting an upgrade to HIGH.  

2. **Escalation Risk**:  
   Previously assessed as MODERATE due to the potential for miscalculation. The confirmed integration of armed FAC into the fishing fleet, combined with the exclusion zone enforcement and the frigate's approach, significantly increases the risk of an incident. The presence of multiple Red assets operating in close proximity to Blue forces raises the likelihood of provocation or unintended escalation, justifying an upgrade to ELEVATED.  

3. **Adversary Intent**:  
   Previously assessed as PROBABLE based on initial evidence of coordinated area denial operations. The new intelligence, including HUMINT on Red coast guard orders, SIGINT on the frigate's approach, and UAV imagery of armed FAC, strongly supports the assessment that Red forces are deliberately coordinating to challenge Blue's freedom of navigation patrol. This warrants an upgrade to LIKELY.  

---

### WATCH ITEMS
• **Red Fast Attack Craft (FAC)**: Monitor for changes in activity or indications of weapon readiness.  
• **RNS Frigate Alpha**: Track its approach and monitor for radar or communication activity that may indicate intent.  
• **Red Coast Guard Cutter Sentinel**: Assess its role in exclusion zone enforcement and potential coordination with other Red assets.  

### RECOMMENDED ACTIONS
1. **ISR Tasking**:  
   - Task UAV Kite-31 to shadow the fishing fleet and monitor FAC activity for indications of intent (e.g., movement patterns, weapon readiness).  
   - Task MPA Lookout-21 to maintain ELINT coverage of RNS Frigate Alpha and monitor for radar or communication activity.  

2. **SIGINT Efforts**:  
   - Prioritize decryption of Red tactical net communications to clarify coordination and intent.  

3. **Force Posture**:  
   - Consider adjusting BNS Resolute's patrol route to maintain FON objectives while avoiding direct confrontation.  
   - Prepare contingency plans for potential escalation, including rules of engagement review and force protection measures.  

The situation is now HIGHLY DYNAMIC, with a clear need to maintain situational awareness and prepare for potential escalation. Immediate focus should be on confirming Red intent and ensuring Blue forces are postured to respond effectively to any provocation.

---------- TextMessage (user) ----------
=== TURN 3 — T0+0006H — NEW INTELLIGENCE RECEIVED ===

CURRENT WORLD STATE:
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario_name": "Cerulean Sea Freedom of Navigation Patrol (SYNTHETIC)",
    "turn": 3,
    "dtg": "T0+0006H"
  },
  "actors": {
    "BLUE": {
      "BNS Resolute (DDG-X1)": {
        "type": "DDG",
        "position": "Grid AA-12",
        "status": "on patrol",
        "mission": "FON transit"
      },
      "MPA Lookout-21": {
        "type": "MPA",
        "position": "Grid AA-15",
        "status": "airborne",
        "mission": "maritime ISR"
      },
      "UAV Kite-31": {
        "type": "UAV",
        "position": "Grid AA-16",
        "status": "airborne",
        "mission": "surface search"
      }
    },
    "RED": {
      "RNS Frigate Alpha (assessed)": {
        "type": "FFG",
        "position": "unknown",
        "status

**ASSESSMENT UPDATE — Turn 3**

| Assessment           | Previous   | Updated   | Δ         | Confidence | Key Evidence                                                                 |
|----------------------|------------|-----------|-----------|------------|------------------------------------------------------------------------------|
| Overall Threat       | HIGH       | CRITICAL  | ↑         | 0.8 → 0.9  | Red forces employing EW, fire-control radar, and planning an exclusion zone; UAV loss reduces ISR capability. |
| Escalation Risk      | ELEVATED   | HIGH      | ↑         | 0.7 → 0.8  | Fire-control radar activity, EW measures, and exclusion zone plans indicate preparation for confrontation. |
| Adversary Intent     | LIKELY     | VERY LIKELY | ↑         | 0.7 → 0.85 | Coordinated actions (EW, radar, cordon plans) strongly suggest deliberate area denial and provocation. |

CHANGELOG:
• **Overall Threat**: HIGH → CRITICAL (conf 0.8 → 0.9)  
  Driver: The combination of EW activity, fire-control radar emissions, and plans to establish an exclusion zone within 12 hours represents a significant escalation. The loss of UAV Kite-31 further degrades Blue's situational awareness, increasing the overall threat level.  

• **Escalation Risk**: ELEVATED → HIGH (conf 0.7 → 0.8)  
  Driver: The use of fire-control radar by Red fast attack craft, coupled with EW interference and exclusion zone enforcement orders, significantly increases the risk of miscalculation or deliberate provocation leading to confrontation.  

• **Adversary Intent**: LIKELY → VERY LIKELY (conf 0.7 → 0.85)  
  Driver: The deliberate use of EW, radar activity, and exclusion zone plans strongly indicate a coordinated effort to deny Blue's freedom of navigation and provoke a response. These actions are consistent with deliberate gray-zone escalation.  

---

### RATIONALE

1. **Overall Threat**:  
   Previously assessed as HIGH due to the confirmed presence of armed fast attack craft, the southbound approach of RNS Frigate Alpha, and exclusion zone enforcement. The new intelligence confirming EW activity, fire-control radar emissions, and plans for an exclusion zone within 12 hours represents a significant escalation. The loss of UAV Kite-31 further reduces Blue's ISR capability, increasing the threat level to CRITICAL.  

2. **Escalation Risk**:  
   Previously assessed as ELEVATED due to the potential for miscalculation. The confirmed use of fire-control radar by Red fast attack craft, coupled with EW interference and exclusion zone plans, significantly increases the likelihood of confrontation. These actions indicate preparation for escalation, justifying an upgrade to HIGH.  

3. **Adversary Intent**:  
   Previously assessed as LIKELY based on evidence of coordinated area denial operations. The deliberate use of EW, radar activity, and exclusion zone plans strongly suggests a coordinated effort to deny Blue's freedom of navigation and provoke a response. This warrants an upgrade to VERY LIKELY.  

---

### WATCH ITEMS
• **Red Fast Attack Craft (FAC)**: Monitor for further radar activity or indications of weapon readiness.  
• **Inner Cordon Plans**: Clarify boundaries, enforcement measures, and ROE for the planned exclusion zone.  
• **Reinforcements from Red Mainland**: Assess composition, capabilities, and timing of additional coast guard cutters.  

### RECOMMENDED ACTIONS
1. **ISR Tasking**:  
   - Task MPA Lookout-21 to maintain ELINT coverage of the fishing fleet and fast attack craft for additional radar or communication activity.  
   - Consider deploying an alternate UAV to replace Kite-31 for ISR coverage of the fishing fleet and surrounding area.  

2. **SIGINT Efforts**:  
   - Prioritize decryption of Red tactical net communications to clarify exclusion zone plans and reinforcements.  

3. **Force Posture**:  
   - Consider repositioning BNS Resolute to maintain FON objectives while avoiding direct confrontation.  
   - Prepare contingency plans for potential escalation, including rules of engagement review and force protection measures.  

4. **Diplomatic Engagement**:  
   - Notify regional partners and allies of the developing situation to coordinate potential responses.  
   - Consider issuing a public statement reaffirming Blue's commitment to freedom of navigation and challenging unlawful exclusion zones.  

The situation is now HIGHLY VOLATILE, with a clear need to maintain situational awareness and prepare for potential escalation. Immediate focus should be on understanding Red's exclusion zone plans and ensuring Blue forces are postured to respond effectively to any provocation.

## Human-in-the-Loop — Commander's Turn

A core principle of responsible AI is keeping a **human in the loop** for consequential decisions. AutoGen makes this straightforward — after the automated agents finish their round-robin cycle, the notebook pauses and waits for user input before proceeding.

The commander reviews the latest SITREP and can:
- **Ask a clarifying question** about any assessment
- **Challenge an assessment** — demand the evidence chain
- **Inject a Commander's Critical Information Requirement (CCIR)**

Edit `commander_query` below and run the cell. A new `RoundRobinGroupChat` with 3 agents (ISR, Assessment, Briefer — no Orchestrator needed) processes the query collaboratively.

> **How the response works:** The commander's question is packaged with the current world state and latest SITREP into a single prompt. The three agents each respond in turn — ISR presents evidence, Assessment evaluates the alternative hypothesis, and the Briefer synthesizes a bottom-line answer. This demonstrates **interactive multi-agent human-AI collaboration** where the human retains full decision authority.
>
> **Azure cost note:** Each commander query triggers 3 additional LLM calls to your [Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/what-is-ai-foundry) endpoint. You can monitor token usage and cost in the [Azure portal](https://learn.microsoft.com/azure/ai-foundry/how-to/costs-plan-manage).
>
> **Learn more:** [Responsible AI principles](https://learn.microsoft.com/azure/ai-services/responsible-use-of-ai-overview) | [Human-AI interaction guidelines](https://learn.microsoft.com/ai/guidelines-human-ai-interaction/)


In [11]:
# ═══════════════════════════════════════════════════════════════
# COMMANDER'S INPUT — Edit this string, then run the cell.
# ═══════════════════════════════════════════════════════════════

commander_query = (
    "I'm not convinced the fishing fleet contains militia vessels. "
    "Walk me through the evidence chain for the fast attack craft identification. "
    "How does our threat assessment change if those are just fishing "
    "boats with unusual hull forms?"
)

# ── Send query to agents ──────────────────────────────────────

latest_sitrep = sitrep_history[-1] if sitrep_history else "No SITREPs generated yet."

context_msg = f"""\
The Commander asks:
"{commander_query}"

Current world state (turn {world_state['meta']['turn']}):
{json.dumps(world_state, indent=2)}

Most recent SITREP:
{latest_sitrep}

Respond to the Commander's question collaboratively:
- ISR / Intel Fusion: present the evidence chain with source reliability grades
- Assessment Agent: explain how the threat assessment changes under the Commander's
  alternative hypothesis (fishing boats, not militia)
- Briefing Agent: synthesize a concise bottom-line answer for the Commander"""

render_commander_box(commander_query)

# AutoGen 0.7: RoundRobinGroupChat for the response team
response_termination = MaxMessageTermination(max_messages=3)
response_team = RoundRobinGroupChat(
    [isr_agent, assessment_agent, briefer],
    termination_condition=response_termination,
)

await Console(response_team.run_stream(task=context_msg))

render_info_box(
    "Edit <code>commander_query</code> above and re-run to ask another question."
)

---------- TextMessage (user) ----------
The Commander asks:
"I'm not convinced the fishing fleet contains militia vessels. Walk me through the evidence chain for the fast attack craft identification. How does our threat assessment change if those are just fishing boats with unusual hull forms?"

Current world state (turn 3):
{
  "synthetic": true,
  "disclaimer": "All data is artificially generated for research and educational purposes only.",
  "meta": {
    "scenario_name": "Cerulean Sea Freedom of Navigation Patrol (SYNTHETIC)",
    "turn": 3,
    "dtg": "T0+0006H"
  },
  "actors": {
    "BLUE": {
      "BNS Resolute (DDG-X1)": {
        "type": "DDG",
        "position": "Grid AA-12",
        "status": "on patrol",
        "mission": "FON transit"
      },
      "MPA Lookout-21": {
        "type": "MPA",
        "position": "Grid AA-15",
        "status": "airborne",
        "mission": "maritime ISR"
      },
      "UAV Kite-31": {
        "type": "UAV",
        "position": "Grid 

## Reset / Cleanup

Clear all runtime state to re-run the demo from scratch.

In [12]:
# Reset world state to initial conditions
world_state["meta"]["turn"] = 0
world_state["meta"]["dtg"] = "T0+0000H"
world_state["events"] = [
    {"dtg": "T0+0000H", "description": "BNS Resolute commences FON patrol leg."}
]
world_state["changelog"] = []

# Clear history
turn_history.clear()
sitrep_history.clear()

# Reset agent conversation memory (async in AutoGen 0.7)
from autogen_core import CancellationToken
_ct = CancellationToken()
for agent in agents:
    await agent.on_reset(_ct)

# Clear common log buffer
clear_logs()

# Close model client connection when fully done (optional)
# await model_client.close()

log_success("State cleared. Ready to re-run from Turn 1.")

LogEntry(level='SUCCESS', message='State cleared. Ready to re-run from Turn 1.', timestamp='2026-02-21 04:31:52', extra={})